In [2]:
:ext OverloadedStrings

In [3]:
-- Will print code points instead of actual glyphs
print "你好世界！"

"\20320\22909\19990\30028\65281"

In [4]:
putStrLn "你好世界！"

你好世界！

In [5]:
do
  putStrLn "再见宇宙！"
  putStrLn "开玩笑的"

再见宇宙！
开玩笑的

In [6]:
import Data.Char

In [7]:
ord '我'

25105

In [8]:
putStrLn [chr 25105]

我

In [9]:
import System.Random

In [10]:
-- generate 0 or 1
randomRIO (0, 1)

0

In [11]:
randomRIO (0x4e00, 0x9fff)

30575

In [12]:
import Control.Monad

In [13]:
(: []) . chr <$> randomRIO (0x4e00, 0x9fff) >>= putStrLn

泊

In [14]:
do
  ord <- randomRIO (0x4e00, 0x9fff)
  let hanzi = (: []) $ chr ord
  putStrLn hanzi

啽

In [15]:
-- Would be easier to write this using a range, e.g. [1..n]
doN :: Int -> (() -> a) -> [a]
doN = aux []
  -- use helper function to make tail recursive
  where aux acc 0 _fn = acc
        aux acc n fn = aux (fn () : acc) (n - 1) fn

In [16]:
doN 5 (const 8)

[8,8,8,8,8]

In [94]:
randomHanzi :: Int -> IO String
-- randomHanzi n = sequenceA $ doN n randomChar 
--   where randomChar () = chr <$> randomRIO (0x4e00, 0x9fff)
randomHanzi n = forM [1..n] $ \_ -> chr <$> randomRIO (0x4e00, 0x9fff)

In [92]:
randomHanzi 10 >>= putStrLn

猿簏靶嘕娱齵顆蜾忠跘

In [96]:
do 
  s <- randomHanzi 8
  forM_ s $ \c -> putStr $ c : " | "

昹 | 蛬 | 祏 | 匜 | 瞜 | 苾 | 籊 | 躦 |

In [207]:
-- Print multicolored hanzi with different font sizes
import IHaskell.Display
import Text.Printf

do 
  s <- randomHanzi 10
  let fontSize i | i `mod` 2 == 0 = 48
                 | otherwise = 36
  let textColor i = case i `mod` 3 of
                    0 -> "red"
                    1 -> "green"
                    2 -> "blue"
  -- how do I split this into two lines?
  let span i = printf "<span style='font-size: %dpx; color: %s'>%c</span>" (fontSize i) (textColor i)
  let spans = zipWith span [0..] s
  printDisplay $ html $ concat spans

孧 苒 旆 砭 欃 唎 鈵 鍁 顏 峘

In [21]:
import Data.Array.IArray

In [139]:
-- type annotation is necessary in order for print to succeed, otherwise you
-- get a type error complaining about an ambiguous type variable
arr :: Array Int Int
arr = listArray (0, 3) [1,2,3,4]

print arr
bounds arr
arr ! 2
arr // [(0, 111), (3, 444)]
arr // [(0, arr ! 3), (3, arr ! 0)]  -- swap elements

array (0,3) [(0,1),(1,2),(2,3),(3,4)]

(0,3)

3

array (0,3) [(0,111),(1,2),(2,3),(3,444)]

array (0,3) [(0,4),(1,2),(2,3),(3,1)]

In [148]:
listArray' :: [a] -> Array Int a
listArray' l = listArray (0, length l - 1) l

colors :: [String]
colors = ["red", "orange", "yellow", "green", "blue", "indigo", "violet"]

colors' :: Array Int String
colors' = listArray' colors

choice :: Array Int e -> IO e
choice arr = do
  i <- randomRIO $ bounds arr
  return $ arr ! i

randomColor :: () -> IO String
randomColor () = choice colors'

randomColors :: Int -> IO [String]
randomColors n = forM [1..n] $ \_ -> randomColor ()

In [75]:
do
  colors <- randomColors 8
  print colors

["red","blue","indigo","indigo","blue","blue","violet","violet"]

In [167]:
-- Print hanzi with random colors
do
  let n = 8
  hanzis <- randomHanzi n
  colors <- randomColors n
  
  let span color hanzi = printf "<span style='color: %s'>%s</span>" color [hanzi]
  let spans = zipWith span colors hanzis
  let div = printf "<div style='font-size: 60px; -webkit-text-stroke: 1px gray'>%s</div>" (concat spans :: String)
  printDisplay $ html div

杆 逓 夼 魭 龈 齞 闵 嘘

In [214]:
shuffle :: Array Int e -> IO (Array Int e)
shuffle arr = shuffle' arr firstIndex
  where (firstIndex, lastIndex) = bounds arr
        shuffle' arr step 
          | step >= lastIndex = return arr
          | otherwise = do            
            index <- randomRIO (step, snd $ bounds arr)
            let arr' = arr // [(step, arr ! index), (index, arr ! step)]
            shuffle' arr' (step + 1)

shuffledColors :: Int -> IO [String]
shuffledColors n = take n . cycle . elems <$> shuffle colors'

shuffledColors 20
shuffledColors 20

["indigo","green","orange","blue","violet","red","yellow","indigo","green","orange","blue","violet","red","yellow","indigo","green","orange","blue","violet","red"]

["red","violet","orange","green","yellow","indigo","blue","red","violet","orange","green","yellow","indigo","blue","red","violet","orange","green","yellow","indigo"]

In [137]:
-- indentation when using guards
coolFn :: Int -> String
coolFn x 
  | x == 1 = "one"
  | x == 2 = "two"
  | x > 2 = 
    "otherwise another string is returned"
      
coolFn 1
coolFn 2
coolFn 3

"one"

"two"

"otherwise another string is returned"

In [178]:
take 8 $ cycle [1,2,3]

[1,2,3,1,2,3,1,2]

In [215]:
-- Print hanzi with shuffled colors
do
  let n = 20
  hanzis <- randomHanzi n
  colors <- shuffledColors n
  
  let span color hanzi = printf "<span style='color: %s'>%s</span>" color [hanzi]
  let spans = zipWith span colors hanzis
  let div = printf "<div style='font-size: 48px; -webkit-text-stroke: 1px gray'>%s</div>" (concat spans :: String)
  printDisplay $ html div

艺 矩 禥 贎 嚠 眷 硎 嬪 詋 駜 酣 虿 礊 烱 哪 塎 弱 吹 湁 灄